<h2>Extraccion de datos de las copias de seguridad de los cursos Moodle  
</h2>

   Moodle proporciona un sistema de logs (informes de actividad y registros),  a nivel de aula virtual, curso, actividad. Estos o bien son interactivos (informes de actividad) o bien son archivos que se presentan en formatos csv; xls-ods. Proporcionan una tabla de registros con los campos _'Curso', 'Hora', 'Enderezo IP', 'Nome completo', 'Acción', 'Información'_. Estos ficheros pueden alcazar fácilmente una longitud de varios millares de líneas. Además los campos propios de Moodle _'Acción', 'Información'_ son a menudo cripticos, como una Id de la base de datos Moodle, una etiqueta de significado poco claro....

   En todo caso, extraer información útil de estos archivos o informes interactivos requiere un trabajo más o menos considerable y cierta capacidad de procesamiento de la información. Debe solicitarse cada archivo o página interactiva que se desee analizar. En todo caso, la repetición de todo este trabajo en varios cursos/varias actividades corre el riesgo de hacerse inasumible. 





Una fuente alternativa de información sobre la actividad de los usuarios es el análisis de las copias de seguridad del curso.  
El administrador el AV puede configurar la creación automática de copias de seguridad, eliminando la necesidad de generar cada copia por demanda. Por defecto las copias de seguridad no incluyen los registros, pero esto puede cambiarse en la configuración de la copia de saguridad (CS). Por defecto la copia de seguridad incluye todas las acciones del usuario que tengan algún tipo de resultado en _assigments_ (Tarefas) y _quiz_ (Probas) y otras _activities_ definidas por Moodle. La copia de seguridad, que es un _dump_ de parte de la base de datos de Moodle en un árbol de archivos _xml_, permite un seguimiento más claro de una parte de las actividades del alumno. 

In [1]:
#Inicialización 



import xml.etree.ElementTree as ET
import re
import time
import glob

PATH='/home/luis/sustituciones/centros/Moodle/AV/'
ANNO=''
SUFF='Salvaterra/'


### Clase MBZ
La clase MBZ es un objeto diseñado para la extracción de información de una copia de seguridad de un curso Moodle.  

Para instanciar una copia de seguridad es necesario proporcionar la ruta a un archivo _mbz_.
    CS=MBZ('pathtofile')
esto hace que se descomprima el archivo _mbz_ en un directorio del mismo nombre y se lea parte de la información:
* .course: ruta a la CS del curso
* .dstudent: diccionario con los usuarios del curso _{ADId:{nome,apelido,rol}}_ Se añade el resultado (_grades_) del usuario en cada _quiz_, en el orden que figura en .list_quiz 
* .dactivs: diccionario con las actividades del curso con al menos un usuario {tipo: [(act_name,dates,users,pth),...]}
     donde _tipo_ en un tipo de actividad Moodle ( _assign, quiz, feedback, hotpotatos, ..._); *act_name* el nombre de la actividad, _dates_ es una lista [None] si no hay fechas aplicables, [fecha_apertura, fecha_cierre] para _quiz_ y [fecha_apertura, fecha_cierre, fecha límite] para _assign_ 
* .name: nombre del curso
* .list_quiz: lista con los _quiz_ del curso




In [2]:
class MBZ():
    
    '''
    Objeto para la extraccion de información de las copias de seguridad de los cursos.
    Extrae información de actividades y usuarios'''
    
    def convert_time(self,strtime):
        '''
        convierte el timestamp en human-readable
        '''
        
        import datetime
        if not strtime:
            return None
        if type(strtime)== str:
            strtime=int(strtime)
        dt=datetime.datetime.fromtimestamp(strtime)

        return dt.strftime("%d/%m/%Y, %H:%M:%S")

    
    def packXml(self,pathtofile):
        '''
        Extrae de forma conveniente el texto XML de los archivos de la CS
        Es necesario dar el path al archivo  xml'''
        try:
            fich=open(pathtofile,'r')
        except:
            return None

        text='\n'.join(fich.readlines())
        fich.close()
        return text


    def extrae_users(self,course):
        '''
        Extrae la información de los usuarios del curso.
            course: path al directorio en el que se extrajo la copia de seguridad
        
        Devuelve un diccionario de diccionarios con la inf. de los usuarios del curso
            {AVId:{nome,apelido,rol}}
        
        '''


        text=self.packXml(course+'/users.xml')

        tree = ET.fromstring(text)

        idents={} #diccionario con la lista de usuarios del grupo {AVId:{nome/apelido/rol}}

        #Carga la lista de usuarios del curso
        for item in tree.iter('user'):
            nome=item.find('firstname').text
            apelido=item.find('lastname').text
            AVId=int(item.get('id'))

            idents[AVId]=idents.get(AVId,{'nome':nome,'apelido':apelido, 'rol':0})

        #Registra el rol de cada usuario del curso
        text=self.packXml(course+'/course/roles.xml')

        tree = ET.fromstring(text)
        for item in tree.iter('assignment'):
            AVId=int(item.find('userid').text)
            rol=int(item.find('roleid').text)
            idents[AVId]['rol']=rol

        return idents

    def extrae_activities(self,course):
           
        '''
        Extrae un diccionario con las actividades del curso
        dactiv[tipo]=(nombre,path_al_xml)'''

        dactiv={}

        for activ in glob.glob(course+'/activities/*'):   
            #Obtener el nombre de la actividad
            suffix=re.findall('([a-z]+)_[0-9]+',activ)[0]
          
            

            text=self.packXml(activ+'/inforef.xml')
            act=ET.fromstring(text)
            users=[item.find('id').text for item in act.findall('userref/user')]
            num_users=len(users)
            
           
                
                    
                
            
            #Si la actividad tiene usuarios
            if num_users:
                

                pth=activ+'/'+suffix+'.xml'
                act=ET.fromstring(self.packXml(pth))
                act_name=act.find(suffix+'/name').text
                dates=[None]
                if suffix=='assign':
                    dates=[act.find(suffix+'/allowsubmissionsfromdate').text,
                           act.find(suffix+'/duedate').text,act.find(suffix+'/cutoffdate').text]
                elif suffix=='quiz' or suffix=='feedback':
                    dates=[act.find(suffix+'/timeopen').text,act.find(suffix+'/timeclose').text]
                #dates=[self.convert_time(item) for item in dates]
                dactiv[suffix]=dactiv.get(suffix,[])+[(act_name,dates,users,pth)]
                #Si existe fichero de logs 
                logs=self.packXml(activ+'/logs.xml')
                if logs:
                    
                    act=ET.fromstring(logs)
                    luser=[]
                    
                    for item in act.findall('log'):
                        
                        userid=item.find('userid').text
                        #print(userid)
                        time=item.find('time').text
                        ip=item.find('ip').text
                        action=item.find('action').text
                        url=item.find('url').text
                        if 'attempt=' in url:
                            url=url.split('=')[1]
                        info=item.find('info').text
                        luser.append((userid,time,action,url,info,ip) )
                    
                    self.logs[act_name]=self.logs.get(act_name,[])+luser
                

        return dactiv



    def extrae_quiz(self,pth):
        '''
        Devuelve un diccionario 
        {usuario: [el mejor resultado en cada quiz del curso]}
        '''

        text=self.packXml(pth)

        tree = ET.fromstring(text)
        dgrad={}
        for item in tree.iter('grade_grade'):
            userid=item.find('userid').text
            grade=item.find('finalgrade').text
            try:
                grade=float(grade)
            except:
                grade=-10.0
            dgrad[userid]=grade
        return dgrad
    
    def extrae_attempts(self,pth):
        '''
        Devuelve un diccionario del quiz con path pth
        
        {AVId:[(grade, to, t1-to,),...]}'''
     
        text=self.packXml(pth)

        tree = ET.fromstring(text)
        
        for item in tree.iter('quiz'):
            total,canon=(float(item.find('sumgrades').text),float(item.find('grade').text))
        coef=canon/total
        #print(total,canon,coef)
        
        dgrad={}
        for item in tree.iter('attempt'):
            attId=int(item.find('uniqueid').text)
            userid=int(item.find('userid').text)
            try:
                grade=float(item.find('sumgrades').text)*coef
            except:
                grade=None
            t0=int(item.find('timestart').text)
            t1=int(item.find('timefinish').text)
            if t1>t0 and not grade==None:
                dgrad[userid]=dgrad.get(userid,[])+[(attId,grade,t0,t1-t0)]
            
        return dgrad
    
    
    def construct_dquiz(self):
        self.list_quiz=[item[0] for item in self.dactivs['quiz']]
        
        for key in self.dstudent.keys():
            self.dstudent[key]['quiz']=[-10]*len(self.list_quiz)
        
        for indx,item in enumerate(self.dactivs['quiz']):
            pth=item[-1]
            #print(pth.replace('quiz.xml','grades.xml'))
            res=self.extrae_quiz(pth.replace('quiz.xml','grades.xml'))
            
            if res:
                res={int(key):value for key,value in res.items()}
                #print(res.items())
                for AVId in map(int,res.keys()):
                    #print(AVId)
                    self.dstudent[AVId]['quiz'][indx]=res[AVId]
                
        
    
    
    def __init__(self,course):
        self.course=course
        self.logs={}
        self.dstudent=self.extrae_users(course)
        self.dactivs=self.extrae_activities(course)
        
        self.name=course.strip().split('/')[-1]
        self.construct_dquiz()
        
    def getName(self):
        return(self.name)
        
    def getStudents(self,rol=5,anonymous=False):
        print('Alumn; '+'; '.join(self.list_quiz))
        for key,val in self.dstudent.items():
            if val['rol']==rol:
                if anonymous:
                    cad=str(key)
                else:
                    cad=', '.join([val['apelido'],val['nome']])
                cad+='; '+'; '.join([str(item) for item in val['quiz']])
                print(cad)
        
    
    def getActivs(self,category=[],anonymous=False):
        '''
        Imprime el diccionario dactivs en forma human-readable
        '''

        if category and not type(category)==list:
            category=[category]
        if not category:
            category=self.dactivs.keys()

        for key in category:
            print('\n',key,'\n')
            for item in self.dactivs[key]:
                print(item[0])
                if item[1]:
                    print('\t',[self.convert_time(it) for it in item[1]])
                print('\tusuarios')
                usuarios=[self.getApNom(int(user)) if not anonymous else user for user in item[2] 
                          if self.dstudent[int(user)]['rol']==5]
                print('\t\tTotal: {} \n\t\t{}'.format(len(usuarios),'; '.join(usuarios)))
    
    def getListQuiz(self):
    
        return self.list_quiz
    
    
    
    
    def getBestQuiz(self,patterns=[],okprint=True):
        
        res={', '.join([value['apelido'],value['nome']]):value['quiz'] 
             for key,value in self.dstudent.items() if value['rol']==5}
        labels=list(self.list_quiz)
        if patterns:
            lquiz=[]
            for pat in patterns:
                for indx,val in enumerate(self.list_quiz):
                    if pat in val:
                        lquiz.append(indx)
            lquiz=tuple(set(lquiz))
            labels=[item for indx,item in enumerate(labels) if indx in lquiz]
            for key in res.keys():
                res[key]=[item for indx,item in enumerate(res[key]) if indx in lquiz]
            
            if okprint:
                print('; '.join(['Apelidos, Nome']+labels))
                for key,value in res.items():
                    print('; '.join([key]+[str(item) for item in value]))
                    
        
        return  labels,res
            
        
    def getApNom(self,AVId):
        return ', '.join([self.dstudent[AVId]['apelido'],self.dstudent[AVId]['nome']])
    
    
        
    def getQuiz(self,quiz=False,okprint=False):
        
        if quiz:
            datt=self.extrae_attempts(self.dactivs['quiz'][quiz][-1])
            if okprint:
                print(self.dactivs['quiz'][quiz][0],datt)
            return datt
        else:
            res=[]
            for quiz in self.dactivs['quiz']:
                datt=self.extrae_attempts(quiz[-1])
                if okprint:
                    print(quiz[0],datt)
                res.append(datt)
            return res
    
    def exportCSV(labels,dic,path=None):
        if path==None:
            print('; '.join(labels))
        

In [3]:
def convert_time(strtime):
        '''
        convierte el timestamp en human-readable
        '''
        
        import datetime
        if not strtime:
            return None
        if type(strtime)== str:
            strtime=int(strtime)
        dt=datetime.datetime.fromtimestamp(strtime)

        return dt.strftime("%d/%m/%Y, %H:%M:%S")
    
def analisis1(data):
    
    import numpy as np
    data=np.array(data)
    return data.size,data.mean(),data.std(),data.min(),np.quantile(data,0.25),np.quantile(data,0.5),np.quantile(data,0.75),data.max()

### Proceso de las copias de seguridad
procesa el lote de copias de seguridad Moodle (*.mbz) situado en el directorio _PATH+SUFF_

In [6]:
%%time
import os,zipfile


for item in glob.glob(PATH+SUFF+'*.mbz'):
    nomes=item.strip().split('/')[-1].split('.')[0]
    with zipfile.ZipFile(item,"r") as zip_ref:
        zip_ref.extractall(PATH+SUFF+nomes)

#seleccion solo de los directorios, nombres sin punto
courses=(set(glob.glob(PATH+SUFF+'*'))-set(glob.glob(PATH+SUFF+'*.*')))

lcourses=[]
lCS=[]
for course in courses:
    lCS.append(MBZ(course))

for indx,item in enumerate(lCS):
    print(indx,'\t',item.getName())

0 	 fpb1-ccaa-15032020-2158
1 	 tecno3eso-b-15032020-2147
2 	 tecno2eso-bcd-15032020-2142
3 	 tic1-b-15032020-2154
4 	 Logs-tecno2eso-bcd-16032020-0246
5 	 tics4eso-bc-15032020-2156
CPU times: user 5.79 s, sys: 1.39 s, total: 7.18 s
Wall time: 16.9 s


Los cursos pueden procesarse por lotes o individualmente

In [8]:
FPB1,ESO3,ESO2,TIC1,LESO2,ESO4=lCS
for CS in lCS:
    print(CS.name)

fpb1-ccaa-15032020-2158
tecno3eso-b-15032020-2147
tecno2eso-bcd-15032020-2142
tic1-b-15032020-2154
Logs-tecno2eso-bcd-16032020-0246
tics4eso-bc-15032020-2156


La lista de _quiz_ y información básica sobre cada actividad son inmediatas  
Para cada actividad se registra su nombre, fecha de apertura da la actividad, fecha de cierre de la actividad, y los usuarios que participaron en la actividad. _MBZ.getActivs()_ imprime con formato la información de _MBZ.dactiv_

In [19]:
print(ESO2.list_quiz)
print(ESO2.getActivs(anonymous=True))

['Repesca:C1- O método de proxectos', 'Repesca: C2-Representación de obxectos', 'Cuestionario: O método de proxectos', 'Ejercicios escalas', 'Cuestionario: Representación de obxectos', 'Cuestionario I. Materiais e propiedades', 'Cuestionario: Materiales & Madera', 'Problemas: Planos y palancas', 'Ejercicion de planos y palancas', 'Ejercicios: Planos y palancas crono']

 forum 

Foro FAQs
	 [None]
	usuarios
		Total: 0 
		

 quiz 

Repesca:C1- O método de proxectos
	 ['23/10/2019, 06:51:00', '31/05/2020, 15:51:00']
	usuarios
		Total: 1 
		2956
Repesca: C2-Representación de obxectos
	 ['25/11/2019, 00:30:00', '31/05/2020, 14:30:00']
	usuarios
		Total: 1 
		2956
Cuestionario: O método de proxectos
	 ['23/10/2019, 06:51:00', '08/11/2019, 15:51:00']
	usuarios
		Total: 62 
		2943; 2944; 2945; 2946; 2947; 2948; 2949; 2950; 2951; 2952; 2953; 2954; 2955; 2956; 2957; 2958; 2959; 2960; 2961; 2962; 2963; 2964; 2965; 2966; 2967; 2968; 2969; 2970; 2971; 2972; 2973; 2974; 2975; 2976; 2977; 2978; 2979;

La calificación (_grade_) de cada alumno en cada quiz también es inmediata

In [20]:
ESO2.getStudents(anonymous=True)

Alumn; Repesca:C1- O método de proxectos; Repesca: C2-Representación de obxectos; Cuestionario: O método de proxectos; Ejercicios escalas; Cuestionario: Representación de obxectos; Cuestionario I. Materiais e propiedades; Cuestionario: Materiales & Madera; Problemas: Planos y palancas; Ejercicion de planos y palancas; Ejercicios: Planos y palancas crono
2943; -10; -10; 7.66667; -10; 4.35898; -10; -10; -10; 1.0; -10
2944; -10; -10; 6.66667; -10; 4.61538; -10; -10; -10; 6.0; 2.0
2945; -10; -10; 6.0; -10; 2.82052; -10; -10; -10; 3.0; -10
2946; -10; -10; 5.5; -10; 3.84615; -10; -10; -10; 3.0; -10
2947; -10; -10; 6.0; -10; -10; -10; -10; -10; -10; -10
2948; -10; -10; 5.33333; -10; 8.20513; -10; -10; -10; 5.0; 10.0
2949; -10; -10; 5.66667; -10; 1.53846; -10; -10; -10; -10; -10
2950; -10; -10; 8.16667; -10; 4.10256; -10; -10; -10; 5.0; -10
2951; -10; -10; 7.0; 5.0; 5.25641; -10; -10; -10; 0.0; -10
2952; -10; -10; 5.16667; -10; 3.84615; -10; -10; -10; -10; -10
2953; -10; -10; 5.83333; -10; 6.6

Pueden procesarse los intentos de cada usuario en cada prueba para obtener (nº intentos, media, desv.tip., min., Q25, Q50, Q75, max) de la calificación y el tiempo empleado por cada usuario en cada prueba

In [21]:
quizdata=ESO2.getQuiz()
for quiz,data in zip(ESO2.list_quiz,quizdata):
    print(quiz)
    print('nº usuarios',len(data.keys()))
    for key,value in data.items():
        grade=[item[1] for item in value]
        time=[item[-1] for item in value]
        print(key,'-->',analisis1(grade),'\n\t',analisis1(time))
    print()

Repesca:C1- O método de proxectos
nº usuarios 1
2956 --> (2, 7.41667, 0.75, 6.66667, 7.04167, 7.41667, 7.79167, 8.16667) 
	 (2, 198.5, 7.5, 191, 194.75, 198.5, 202.25, 206)

Repesca: C2-Representación de obxectos
nº usuarios 1
2956 --> (1, 2.307692307692308, 0.0, 2.307692307692308, 2.307692307692308, 2.307692307692308, 2.307692307692308, 2.307692307692308) 
	 (1, 543.0, 0.0, 543, 543.0, 543.0, 543.0, 543)

Cuestionario: O método de proxectos
nº usuarios 62
2943 --> (2, 6.66667, 1.0, 5.66667, 6.16667, 6.66667, 7.16667, 7.66667) 
	 (2, 294.0, 19.0, 275, 284.5, 294.0, 303.5, 313)
2944 --> (2, 5.66667, 1.0, 4.66667, 5.16667, 5.66667, 6.16667, 6.66667) 
	 (2, 283.5, 122.5, 161, 222.25, 283.5, 344.75, 406)
2945 --> (2, 6.0, 0.0, 6.0, 6.0, 6.0, 6.0, 6.0) 
	 (2, 181.0, 14.0, 167, 174.0, 181.0, 188.0, 195)
2946 --> (2, 4.833335, 0.6666650000000001, 4.16667, 4.5000025, 4.833335, 5.1666675, 5.5) 
	 (2, 233.5, 51.5, 182, 207.75, 233.5, 259.25, 285)
2947 --> (1, 6.0, 0.0, 6.0, 6.0, 6.0, 6.0, 6.0) 


O las fechas de cada intento

In [22]:
for quiz,data in zip(ESO2.list_quiz,quizdata):
    print(quiz)
    print('nº usuarios',len(data.keys()))
    for key,value in data.items():
        date=[convert_time(item[2]) for item in value]
        
        print(key,'-->',date)
    print()

Repesca:C1- O método de proxectos
nº usuarios 1
2956 --> ['06/03/2020, 11:55:36', '11/03/2020, 09:27:06']

Repesca: C2-Representación de obxectos
nº usuarios 1
2956 --> ['11/03/2020, 09:59:11']

Cuestionario: O método de proxectos
nº usuarios 62
2943 --> ['25/10/2019, 11:38:01', '30/10/2019, 09:35:58']
2944 --> ['25/10/2019, 11:55:36', '30/10/2019, 10:01:29']
2945 --> ['25/10/2019, 11:51:37', '30/10/2019, 09:41:15']
2946 --> ['30/10/2019, 09:49:42', '06/11/2019, 09:37:09']
2947 --> ['30/10/2019, 09:41:26']
2948 --> ['25/10/2019, 12:03:38', '30/10/2019, 09:55:41']
2949 --> ['23/10/2019, 09:32:30', '25/10/2019, 11:42:21']
2950 --> ['25/10/2019, 11:39:03', '30/10/2019, 09:38:24']
2951 --> ['23/10/2019, 09:34:28', '25/10/2019, 11:51:56']
2952 --> ['30/10/2019, 09:41:40', '06/11/2019, 09:47:44']
2953 --> ['25/10/2019, 11:48:54', '30/10/2019, 09:34:31']
2954 --> ['25/10/2019, 11:43:12', '30/10/2019, 09:46:34']
2955 --> ['25/10/2019, 11:38:46']
2956 --> ['25/10/2019, 11:42:29', '30/10/2019, 0

Si la copia de seguridad contiene los registros se procesan automaticamente

In [23]:
ESO2.logs

{}

In [24]:
LESO2.logs.keys()


dict_keys(['Foro FAQs', 'Repesca:C1- O método de proxectos', 'Repesca: C2-Representación de obxectos', 'Repesca: Resumo do 1º Trimestre', 'O procesador de textos', 'Calificación do exemplo', 'Cuestionario: O método de proxectos', 'Memoria de proxecto 0', 'Ejercicios escalas', 'Os_metais.jqz', 'Resumo do 1º Trimestre', 'Cuestionario: Representación de obxectos', 'Cuestionario I. Materiais e propiedades', 'Resumo do 2º Trimestre', 'Cuestionario: Materiales & Madera', 'Problemas: Planos y palancas', 'Ejercicion de planos y palancas', 'Ejercicios: Planos y palancas crono'])

In [25]:
print('Logs de {} ({})'.format('Ejercicion de planos y palancas',len(LESO2.logs['Ejercicion de planos y palancas'])))
print()
print('AVId; data; action; url; info; ip')
for item in LESO2.logs['Ejercicion de planos y palancas']:
    if LESO2.dstudent[int(item[0])]['rol']==5:
        print(item[0],'; ',convert_time(item[1]),'; ','; '.join(item[2:]))

Logs de Ejercicion de planos y palancas (1464)

AVId; data; action; url; info; ip
2951 ;  19/02/2020, 09:30:45 ;  view; view.php?id=18588; 1391; 10.66.194.117
2951 ;  19/02/2020, 09:31:15 ;  attempt; 17442; 1391; 10.66.194.117
2951 ;  19/02/2020, 09:31:22 ;  continue attempt; 17442; 1391; 10.66.194.117
2948 ;  19/02/2020, 09:36:40 ;  view; view.php?id=18588; 1391; 10.66.194.128
3091 ;  19/02/2020, 09:37:13 ;  view; view.php?id=18588; 1391; 10.66.194.118
3091 ;  19/02/2020, 09:37:51 ;  attempt; 17443; 1391; 10.66.194.118
3091 ;  19/02/2020, 09:37:52 ;  continue attempt; 17443; 1391; 10.66.194.118
2957 ;  19/02/2020, 09:39:02 ;  view; view.php?id=18588; 1391; 10.66.194.115
2996 ;  19/02/2020, 09:39:14 ;  view; view.php?id=18588; 1391; 10.66.194.109
2948 ;  19/02/2020, 09:39:35 ;  attempt; 17444; 1391; 10.66.194.128
2948 ;  19/02/2020, 09:39:35 ;  continue attempt; 17444; 1391; 10.66.194.128
2957 ;  19/02/2020, 09:40:36 ;  view; view.php?id=18588; 1391; 10.66.194.115
2957 ;  19/02/2020, 0

2991 ;  09/03/2020, 13:57:57 ;  continue attempt; 17721; 1391; 10.66.194.105
2983 ;  09/03/2020, 13:57:59 ;  view summary; 17651; 1391; 10.66.194.117
2983 ;  09/03/2020, 13:58:10 ;  close attempt; 17651; 1391; 10.66.194.117
2983 ;  09/03/2020, 13:58:11 ;  review; 17651; 1391; 10.66.194.117
2983 ;  09/03/2020, 13:58:33 ;  review; 17651; 1391; 10.66.194.117
2991 ;  09/03/2020, 13:59:40 ;  continue attempt; 17721; 1391; 10.66.194.105
2991 ;  09/03/2020, 13:59:43 ;  view summary; 17721; 1391; 10.66.194.105
2991 ;  09/03/2020, 13:59:48 ;  close attempt; 17721; 1391; 10.66.194.105
2991 ;  09/03/2020, 13:59:48 ;  review; 17721; 1391; 10.66.194.105
2973 ;  09/03/2020, 15:15:02 ;  view; view.php?id=18588; 1391; 91.116.163.117
2973 ;  09/03/2020, 15:15:05 ;  continue attempt; 17717; 1391; 91.116.163.117
2973 ;  09/03/2020, 15:15:42 ;  continue attempt; 17717; 1391; 91.116.163.117
2988 ;  09/03/2020, 15:42:25 ;  view; view.php?id=18588; 1391; 88.31.106.250
2988 ;  09/03/2020, 15:42:28 ;  continue